In [1]:
import ipynb
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_squeeze_data, split_info
from ipynb.fs.full.train_model import train_model, accuracy_check

dat, description = load_subject(1)
labels = extract_labels(description)
data = extract_action_interval(dat)
train_data, val_data, test_data, train_labels, val_labels, test_labels = split_squeeze_data(data, labels)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 


# Shallow CNN
Implementation of the shallow CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
import copy

class ShallowCNN(nn.Module):
    def __init__(self, interval = "full"):
        super(ShallowCNN, self).__init__()
        first_channels = 40
        

        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 25), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = 40, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm = nn.BatchNorm2d(40, False)
        # ELU
        self.elu = nn.ELU()
        # Mean pooling
        self.meanpool = nn.AvgPool2d(kernel_size = (1,75), stride = (1,15)) # This 15 might be a 75 :(

        # Classifier
        if interval == "action":
            self.classifier = nn.Linear(1440,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(2840,4, bias = False)
        # Softmax
        self.softmax = nn.Softmax()

        
    def forward(self, x, dropout = 0.5):
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm(res)
        res = self.elu(res)
        res = self.meanpool(res)
        res = F.dropout(res, dropout)
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


In [3]:
network = ShallowCNN(interval = "action").float()
accuracy_check(train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 5, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(train_data, train_labels)

torch.Size([1, 1, 128, 636])
torch.Size([1, 4])


/tmp/ipykernel_53502/2210671637.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


ACCURACY: 0.2777777777777778
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  -9.820293426513672 	 -0.24999761581420898
1 	  -11.118707656860352 	 -0.2499803602695465
2 	  -11.938577651977539 	 -0.2499726563692093
3 	  -12.943338394165039 	 -0.25035560131073
4 	  -13.43875503540039 	 -0.2524065673351288
ACCURACY: 0.5075757575757576


In [4]:
accuracy_check(train_data, train_labels)
accuracy_check(val_data, val_labels)
accuracy_check(test_data, test_labels)

/tmp/ipykernel_53502/2210671637.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)


ACCURACY: 0.48737373737373735
ACCURACY: 0.12962962962962962
ACCURACY: 0.34
